## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Medical Text Translation (EN-ES)](https://aws.amazon.com/marketplace/pp/prodview-lwpnc66c6bcug).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Medical Text Translation (EN-ES)

- **Model**: `jsl-medical-translation-en-es`
- **Model Description**: Medical text translation between English (EN) and Spanish (ES).


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [12]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [13]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
model_name = "jsl-medical-translation-en-es"

real_time_inference_instance_type = "ml.g5.2xlarge"
batch_transform_inference_instance_type = "ml.g5.2xlarge"

## 2. create a deployable model from the model package.

In [15]:
# Define ModelPackage

model = ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```


#### Important Parameters

- direction: str
   - Allowed values: ["en_to_es", "es_to_en"]
   - Default: "en_to_es"
You can specify these parameters in the input as follows:
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "direction": "en_to_es"
}
```

### 2.1 Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

#### A. Deploy the SageMaker model to an endpoint

In [16]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

-----------!

Once endpoint has been created, you would be able to perform real-time inference.

In [17]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

#### Initial Setup

In [18]:
english_text = "Primary empty sella turcica associated with diabetes insipidus and campimetric defect"
english_records = [
    "The patient was diagnosed with acute myocardial infarction.",
    "Chronic obstructive pulmonary disease is characterized by long-term breathing problems.",
]


spanish_text = "Evaluación clínico-radiológica de la técnica de aloinjerto compactado y vástago cementado en cirugías de revisión de cadera."
spanish_records = [
    "El paciente fue sometido a una tomografía computarizada del abdomen.",
    "La hipertensión arterial es una condición crónica que puede causar daño a varios órganos."
]

#### JSON

##### Example 1

In [21]:
input_json_data = {"text": english_text, "direction": "en_to_es"}

response_json =  invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json")

pd.DataFrame(response_json)

,predictions
0,Sele turcica primaria vacía asociada con diabetes insípida y defecto campimétrico


##### Example 2

In [24]:
input_json_data = {"text": spanish_text, "direction": "es_to_en"}

response_json =  invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json")

pd.DataFrame(response_json)

,predictions
0,Radiologic evaluation of the compacted allograft and cemented stem technique in hip revision surgeries.


#### JSON Lines

In [36]:
import json

def create_jsonl(records, direction):
    if isinstance(records, str):
        records = [records]
    json_lines = '\n'.join(json.dumps({"text": text, "direction": direction}, ensure_ascii=False) for text in records)

    return json_lines


##### Example 1

In [27]:
input_jsonl_data = create_jsonl(english_text, direction= "en_to_es")

response_json =  invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines")

print(response_json)

{"predictions": "Sele turcica primaria vacía asociada con diabetes insípida y defecto campimétrico"}


##### Example 2

In [30]:
input_jsonl_data = create_jsonl(spanish_text, direction= "es_to_en")

response_json =  invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines")

print(response_json)

{"predictions": "Radiologic evaluation of the compacted allograft and cemented stem technique in hip revision surgeries."}


#### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [32]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

### 2.2 Batch inference

In [37]:
validation_json_file_name1 = "input1.json"
validation_json_file_name2 = "input2.json"

validation_jsonl_file_name1 = "input1.jsonl"
validation_jsonl_file_name2 = "input2.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [38]:
# Example data for JSONL files
input_jsonl_data1 = create_jsonl(english_records, direction= "en_to_es")
input_jsonl_data2 = create_jsonl(spanish_records, direction= "es_to_en")

# Example data for JSON files
input_json_data1 = json.dumps({"text": english_records, "direction": "en_to_es"}, ensure_ascii=False)
input_json_data2 = json.dumps({"text": spanish_records, "direction": "es_to_en"}, ensure_ascii=False)

# Write and upload JSONL data
upload_to_s3(input_jsonl_data1, validation_jsonl_file_name1)
upload_to_s3(input_jsonl_data2, validation_jsonl_file_name2)

# Write and upload JSON data
upload_to_s3(input_json_data1, validation_json_file_name1)
upload_to_s3(input_json_data2,  validation_json_file_name2)


#### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [45]:
# EN ==> ES
retrieve_json_output_from_s3(validation_json_file_name1)

{'predictions': ['Se diagnosticó al paciente un infarto agudo de miocardio.',
  'La enfermedad pulmonar obstructiva crónica se caracteriza por problemas respiratorios a largo plazo.']}

In [47]:
# ES ==> EN
retrieve_json_output_from_s3(validation_json_file_name2)

{'predictions': ['The patient underwent a CT scan of the abdomen.',
  'Hypertension is a chronic condition that can cause damage to several organs.']}

#### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [ ]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [51]:
# EN ==> ES
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name1)

{"predictions": "Se diagnosticó al paciente un infarto agudo de miocardio."}
{"predictions": "La enfermedad pulmonar obstructiva crónica se caracteriza por problemas respiratorios a largo plazo."}


In [52]:
# ES ==> EN
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name2)

{"predictions": "The patient underwent a CT scan of the abdomen."}
{"predictions": "Hypertension is a chronic condition that can cause damage to several organs."}


In [25]:
model.delete_model()

INFO:sagemaker:Deleting model with name: jsl-medical-translation-en-es-1-2024-09-13-14-18-21-133


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

